In [83]:
import pandas as pd
import requests
import json
from config import gkey
import gmaps

In [84]:
gmaps.configure(api_key=gkey)

In [85]:
# pulling in the data from weatherpy
weather_url = "../weatherpy/results/weather_data.csv"

In [86]:
weather = pd.read_csv(weather_url)

In [87]:
weather.head()

,city,latitude,longitude,temperature,humidity,cloudiness,wind,datetime,country
0,punta arenas,-53.1500,-70.9167,48.09,66,75,25.32,1611948215,CL
1,kirovskiy,45.0917,133.5136,-23.69,86,4,2.57,1611948216,RU
2,san cristobal,7.7669,-72.2250,84.20,58,40,13.80,1611947928,VE
3,maningrida,-12.0667,134.2667,76.30,91,81,6.11,1611948218,AU
4,necochea,-38.5473,-58.7368,78.01,42,35,6.80,1611948219,AR


In [88]:
weather.dtypes

city            object
latitude       float64
longitude      float64
temperature    float64
humidity         int64
cloudiness       int64
wind           float64
datetime         int64
country         object
dtype: object

In [89]:
#data for heatmap

locations = weather[["latitude", "longitude"]]
humidity = weather["humidity"].astype(float)

In [90]:
#heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [91]:
#choosing vacation spots

vacation_spots = weather.loc[(weather["temperature"]>77)&
            (weather["temperature"]<83)&
            (weather["cloudiness"]<20)&
            (weather["wind"]<10)&
            (weather["humidity"]>40)&
            (weather["humidity"]<80),:]

vacation_spots["hotel"]=""

vacation_spots

<ipython-input-91-90dfc5f80fd2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_spots["hotel"]=""


,city,latitude,longitude,temperature,humidity,cloudiness,wind,datetime,country,hotel
271,freetown,8.4840,-13.2299,79.56,74,0,8.39,1611948436,SL,
304,kavaratti,10.5669,72.6420,80.51,73,1,9.42,1611948567,IN,
421,caravelas,-17.7125,-39.2481,81.30,68,0,9.31,1611948706,BR,
466,karratha,-20.7377,116.8463,81.52,76,12,4.07,1611948762,AU,
554,grand gaube,-20.0064,57.6608,81.00,77,7,5.99,1611948876,MU,
565,saint-joseph,-21.3667,55.6167,77.88,69,0,4.61,1611948890,RE,


In [92]:
#test api call with one city to decipher the json

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

location = "66.045,-17.338"

params = {
    "location": location,
    "radius": "10000",
    "keyword": "hotel",
    "key": gkey,
}

response = requests.get(url, params=params).json()

In [93]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 66.0460305,
                    "lng": -17.3397434
                },
                "viewport": {
                    "northeast": {
                        "lat": 66.04740452989273,
                        "lng": -17.33827452010728
                    },
                    "southwest": {
                        "lat": 66.04470487010728,
                        "lng": -17.34097417989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Fosshotel H\u00fasav\u00edk",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 427,
                    "html_attributions": [
                 

In [94]:
#hotel name
response["results"][0]["name"]

'Fosshotel Húsavík'

In [95]:
#running the real api call for my vacation spots

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in vacation_spots.iterrows():
    
    location = f"{row['latitude']}, {row['longitude']}"
    
    params = {
    "location": location,
    "radius": "10000",
    "keyword": "hotel",
    "key": gkey}
    
    response = requests.get(url, params=params).json()
    
    vacation_spots.loc[index, "hotel name"] = response["results"][0]["name"]
    

/Users/leah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/leah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [96]:
vacation_spots

,city,latitude,longitude,temperature,humidity,cloudiness,wind,datetime,country,hotel,hotel name
271,freetown,8.4840,-13.2299,79.56,74,0,8.39,1611948436,SL,,"Radisson Blu Mammy Yoko Hotel, Freetown"
304,kavaratti,10.5669,72.6420,80.51,73,1,9.42,1611948567,IN,,Bismillah Hotel And Lodging
421,caravelas,-17.7125,-39.2481,81.30,68,0,9.31,1611948706,BR,,Abrolhos Dive Inn
466,karratha,-20.7377,116.8463,81.52,76,12,4.07,1611948762,AU,,ibis Styles Karratha
554,grand gaube,-20.0064,57.6608,81.00,77,7,5.99,1611948876,MU,,Sea Resorts Hotels Head Office
565,saint-joseph,-21.3667,55.6167,77.88,69,0,4.61,1611948890,RE,,Palm Hotel & Spa


In [97]:
# adding hotels to heatmap
# setting marker locations
marker_locations = vacation_spots[['latitude', 'longitude']]

marker_locations

,latitude,longitude
271,8.4840,-13.2299
304,10.5669,72.6420
421,-17.7125,-39.2481
466,-20.7377,116.8463
554,-20.0064,57.6608
565,-21.3667,55.6167


In [98]:
#setting up info box content in a list

marker_box_list = []

for index, row in vacation_spots.iterrows():
    marker_box = {"Hotel":row["hotel name"], "City":row["city"], "Country":row["country"]}
    marker_box_list.append(marker_box)

In [99]:
marker_box_list

[{'Hotel': 'Radisson Blu Mammy Yoko Hotel, Freetown',
  'City': 'freetown',
  'Country': 'SL'},
 {'Hotel': 'Bismillah Hotel And Lodging',
  'City': 'kavaratti',
  'Country': 'IN'},
 {'Hotel': 'Abrolhos Dive Inn', 'City': 'caravelas', 'Country': 'BR'},
 {'Hotel': 'ibis Styles Karratha', 'City': 'karratha', 'Country': 'AU'},
 {'Hotel': 'Sea Resorts Hotels Head Office',
  'City': 'grand gaube',
  'Country': 'MU'},
 {'Hotel': 'Palm Hotel & Spa', 'City': 'saint-joseph', 'Country': 'RE'}]

In [102]:
# Found this template that made it look way better at https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
# otherwise it was just a string

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**hotel) for hotel in marker_box_list]


In [103]:
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
                             
fig.add_layer(markers)
                             
fig

Figure(layout=FigureLayout(height='420px'))